In [98]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel, \
                        AutoModelForSeq2SeqLM, BartForConditionalGeneration, AutoConfig, pipeline
                        
import torch
from model.prediction import MultiLabelClassifier, AspectBasedSentimentModel

import pandas as pd
import numpy as np

In [2]:
ENTITY_MODEL = './model/multilabel_entity_distil_bert_7epochs_lr5e-5_d&c_dataset'
ASPECT_MODEL_1 = './model/multilabel_aspect_distil_4epochs_lr3e-5_without_test_set_split_keep_same_sent_together.pth'
ASPECT_MODEL_2 = './model/aspect_model_seq2seq_facebooklargebart_lr5e-5_epochs5_w_additional_val_acc_4683'
SENTIMENT_MODEL_1 = './model/sentiment_model_val_acc_6162_lr4.5e-5_wtdecay_1e-4_epochs4_256_256_256_256_smoothed_weight_warmup_and_reducelr_freeze4layers.pth'
SENTIMENT_MODEL_2 = './model/sentiment_model_lr3e-5_epochs5_features_in_text_val_acc_6771'

In [35]:
entity_idx_map = {k:v for v, k in enumerate(['kamala', 'trump', 'others'])}
idx_entity_map = {v:k for k, v in entity_idx_map.items()}
aspect_idx_map = {k:v for v, k in enumerate(['campaign', 'communication', 'competence', 'controversies',
       'ethics and integrity', 'leadership', 
       'personality trait', 'policies', 'political ideology',
       'public image', 'public service record',
       'relationships and alliances', 'voter sentiment', 'others'])}
idx_aspect_map = {v:k for k, v in aspect_idx_map.items()}
sentiment_idx_map = {k:v for v, k in enumerate(['negative', 'neutral', 'positive'])}
idx_sentiment_map = {v:k for k, v in sentiment_idx_map.items()}
idx_sentiment_map2 = {v-1:k for k, v in sentiment_idx_map.items()}

# Entity Model Upload

In [20]:
# Entity Model
model_name = "distilbert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(ENTITY_MODEL)
model.config.label2id = {'Kamala': 0, 'Trump': 1, 'Others': 2}
model.config.id2label = {0: 'Kamala', 1:'Trump', 2:'Others'}

c:\Users\user\Desktop\Projects\NUS-ISS\text\nusiss-project-presidential-absa-system\myVenv\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\user\Desktop\Projects\NUS-ISS\text\nusiss-project-presidential-absa-system\myVenv\lib\site-packages\transformers\modeling_utils.py:399: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbi

In [22]:
# test = pipeline('text-classification', model=model, tokenizer=tokenizer)
# test('hello there kamala.')

[{'label': 'Kamala', 'score': 0.8001327514648438}]

In [21]:
# model.push_to_hub('nusiss-election-project-entity-model-distilbert-base-cased')
# tokenizer.push_to_hub('nusiss-election-project-entity-model-distilbert-base-cased')

pytorch_model.bin: 100%|██████████| 263M/263M [00:37<00:00, 7.10MB/s] 


CommitInfo(commit_url='https://huggingface.co/destonedbob/nusiss-election-project-entity-model-distilbert-base-cased/commit/9ed47239d08ec4ede1d4eb5e4378984baf052ff2', commit_message='Upload tokenizer', commit_description='', oid='9ed47239d08ec4ede1d4eb5e4378984baf052ff2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/destonedbob/nusiss-election-project-entity-model-distilbert-base-cased', endpoint='https://huggingface.co', repo_type='model', repo_id='destonedbob/nusiss-election-project-entity-model-distilbert-base-cased'), pr_revision=None, pr_num=None)

In [ ]:
del model
del tokenizer

# Entity Model Call Test

In [30]:
model_name = 'destonedbob/nusiss-election-project-entity-model-distilbert-base-cased'
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model.to('cuda')

def get_probabilities(texts, score=False):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    inputs.to('cuda')
    outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.sigmoid(logits)
    if not score:
        return np.array(list(map(lambda x: 1 if x > 0.65 else 0, probabilities.cpu().detach().numpy()[0].tolist())))
    else:
        return probabilities.cpu().detach().numpy()[0]

c:\Users\user\Desktop\Projects\NUS-ISS\text\nusiss-project-presidential-absa-system\myVenv\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\user\Desktop\Projects\NUS-ISS\text\nusiss-project-presidential-absa-system\myVenv\lib\site-packages\transformers\modeling_utils.py:399: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbi

In [39]:
test_df = pd.DataFrame(['Is that orange asshole trump?',
                'Kamala is always laughing, incompentent',
                'Joe Biden is a senile old fuck.'], columns=['sentence'])

test_df['sentence'].apply(get_probabilities).apply(lambda x: idx_entity_map[list(x).index(1)].strip())

0     trump
1    kamala
2    others
Name: sentence, dtype: object

# Aspect Model 1 (Not uploaded to Huggingface because model was not inherited from HF models)
Try upload to Github LFS

In [40]:
# Aspect Model 1
model_name = 'distilbert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
num_aspects = 13
model = MultiLabelClassifier(num_labels=num_aspects).to('cuda')
model.load_state_dict(torch.load(ASPECT_MODEL_1))
model.to('cuda')


c:\Users\user\Desktop\Projects\NUS-ISS\text\nusiss-project-presidential-absa-system\myVenv\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassificat

MultiLabelClassifier(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin

# Aspect Model 2 Upload

In [53]:
# Aspect Model 2
model_name = "facebook/bart-large" 
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(ASPECT_MODEL_2)
# model.to('cuda')

c:\Users\user\Desktop\Projects\NUS-ISS\text\nusiss-project-presidential-absa-system\myVenv\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\user\Desktop\Projects\NUS-ISS\text\nusiss-project-presidential-absa-system\myVenv\lib\site-packages\transformers\modeling_utils.py:399: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbi

In [54]:
test = pipeline('text2text-generation', model=model, tokenizer=tokenizer)

In [57]:
test('entity of interest: Trump [SEP] I think Trump is a criminal')

[{'generated_text': 'controversies'}]

In [58]:
# model.push_to_hub('nusiss-election-project-aspect-seq2seq-model-facebook-bart-large')
# tokenizer.push_to_hub('nusiss-election-project-aspect-seq2seq-model-facebook-bart-large')

pytorch_model.bin: 100%|██████████| 1.63G/1.63G [02:59<00:00, 9.05MB/s]


CommitInfo(commit_url='https://huggingface.co/destonedbob/nusiss-election-project-aspect-seq2seq-model-facebook-bart-large/commit/ad3bc9969c29a30ea7ec4f65e50790050f241ace', commit_message='Upload tokenizer', commit_description='', oid='ad3bc9969c29a30ea7ec4f65e50790050f241ace', pr_url=None, repo_url=RepoUrl('https://huggingface.co/destonedbob/nusiss-election-project-aspect-seq2seq-model-facebook-bart-large', endpoint='https://huggingface.co', repo_type='model', repo_id='destonedbob/nusiss-election-project-aspect-seq2seq-model-facebook-bart-large'), pr_revision=None, pr_num=None)

# Aspect Model 2 Call Test

In [60]:
model_name = 'destonedbob/nusiss-election-project-aspect-seq2seq-model-facebook-bart-large'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
pipeline = pipeline('text2text-generation', model=model, tokenizer=tokenizer)

In [70]:
df = pd.DataFrame([
    'entity of interest: trump [SEP] I think Trump is a criminal',
    'entity of interest: trump [SEP] I think Trump has lousy ideas when it comes to the economy',
    'entity of interest: kamala [SEP] Kamala cannot run a country, all she does is laugh', 
    'entity of interest: neither trump nor kamala [SEP] Biden did not make any sense during his debate'
    ], columns=['sentence'])

pipeline(df.sentence.tolist(), batch_size=2)

[{'generated_text': 'controversies'},
 {'generated_text': 'policies'},
 {'generated_text': 'competence'},
 {'generated_text': 'communication'}]

# Sentiment Model 1 (Cannot Upload to Huggingface because model was not written in the framework)

In [74]:
# Sentiment Model 1
model_name = "distilbert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = torch.load('./model/sentiment_model_val_acc_6162_lr4.5e-5_wtdecay_1e-4_epochs4_256_256_256_256_smoothed_weight_warmup_and_reducelr_freeze4layers.pth')
model.to('cuda')

C:\Users\user\AppData\Local\Temp\ipykernel_9984\4293192402.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('./model/sentiment_model_val_acc_6162_lr4.5

AspectBasedSentimentModel(
  (bert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1

# Sentiment Model 2 Upload

In [93]:
# Sentiment Model 2
model_name = "facebook/bart-large" 
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(SENTIMENT_MODEL_2)
pipeline = pipeline('text2text-generation', model=model, tokenizer=tokenizer)

c:\Users\user\Desktop\Projects\NUS-ISS\text\nusiss-project-presidential-absa-system\myVenv\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\user\Desktop\Projects\NUS-ISS\text\nusiss-project-presidential-absa-system\myVenv\lib\site-packages\transformers\modeling_utils.py:399: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbi

In [94]:
# model.push_to_hub('nusiss-election-project-sentiment-seq2seq-model-facebook-bart-large')
# tokenizer.push_to_hub('nusiss-election-project-sentiment-seq2seq-model-facebook-bart-large')

CommitInfo(commit_url='https://huggingface.co/destonedbob/nusiss-election-project-sentiment-seq2seq-model-facebook-bart-large/commit/2c1d32f73fc6b7ad95ccc6ee3652bf2f9183c133', commit_message='Upload tokenizer', commit_description='', oid='2c1d32f73fc6b7ad95ccc6ee3652bf2f9183c133', pr_url=None, repo_url=RepoUrl('https://huggingface.co/destonedbob/nusiss-election-project-sentiment-seq2seq-model-facebook-bart-large', endpoint='https://huggingface.co', repo_type='model', repo_id='destonedbob/nusiss-election-project-sentiment-seq2seq-model-facebook-bart-large'), pr_revision=None, pr_num=None)

# Sentiment Model 2 Call Test

In [88]:
del model 
del tokenizer
del pipeline

In [100]:
model = AutoModelForSeq2SeqLM.from_pretrained('destonedbob/nusiss-election-project-sentiment-seq2seq-model-facebook-bart-large')
tokenizer = AutoTokenizer.from_pretrained('destonedbob/nusiss-election-project-sentiment-seq2seq-model-facebook-bart-large')
my_pipeline = pipeline('text2text-generation', model=model, tokenizer=tokenizer)

c:\Users\user\Desktop\Projects\NUS-ISS\text\nusiss-project-presidential-absa-system\myVenv\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\user\Desktop\Projects\NUS-ISS\text\nusiss-project-presidential-absa-system\myVenv\lib\site-packages\transformers\modeling_utils.py:399: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbi

In [104]:
df = pd.DataFrame([
    'entity of interest: trump [SEP] aspect of interest: controversies [SEP] I think Trump is a criminal',
    'entity of interest: trump [SEP] aspect of interest: policies [SEP] I think Trump has lousy ideas when it comes to the economy',
    'entity of interest: kamala [SEP] aspect of interest: competence [SEP] Kamala cannot run a country, all she does is laugh', 
    'entity of interest: neither trump nor kamala [SEP] aspect of interest: communication [SEP] Biden did not make any sense during his debate', 
    'entity of interest: kamala [SEP] aspect of interest: competence [SEP] Kamala is a really intelligent woman'
    ], columns=['sentence'])

my_pipeline(df.sentence.tolist(), batch_size=5)

[{'generated_text': 'sentiment: negative'},
 {'generated_text': 'sentiment: negative'},
 {'generated_text': 'sentiment: negative'},
 {'generated_text': 'sentiment: negative'},
 {'generated_text': 'sentiment: positive'}]